append-SMB-ATL11
================

Interpolates modeled firn estimates to the coordinates of an ATL11 file

#### Python Dependencies
- [numpy: Scientific Computing Tools For Python](https://numpy.org)  
- [netCDF4: Python interface to the netCDF C library](https://unidata.github.io/netcdf4-python/netCDF4/index.html)  
- [h5py: Python interface for Hierarchal Data Format 5 (HDF5)](https://www.h5py.org/)  
- [pointCollection: Utilities for organizing and manipulating point data](https://github.com/SmithB/pointCollection)  

This notebook uses Jupyter widgets to set parameters for calculating the firn estimates.  
The widgets can be installed as described below.  
```
pip3 install --user ipywidgets
jupyter nbextension install --user --py widgetsnbextension
jupyter nbextension enable --user --py widgetsnbextension
jupyter-notebook
```

#### Load modules 

In [ ]:
import os
import re
import SMBcorr
import h5py
import numpy as np
import pointCollection as pc
import ipywidgets as widgets
import matplotlib.pyplot as plt

#### Choose parameters to run
- Input ATL11 file  
- Directory containing SMB models  
- SMB region to run  
- SMB model to run  

In [ ]:
# choose file to read and append
value=''
inputText = widgets.Text(
    value=value,
    description='File:',
    disabled=False
)

# text menu to set model base directory
value=os.getcwd()
directoryText = widgets.Text(
    value=value,
    description='Directory:',
    disabled=False
)

# dropdown menu for setting region
regionDropdown = widgets.Dropdown(
    options=['AA','GL'],
    value='GL',
    description='Region:',
    disabled=False,
)

# dropdown menu for setting SMB model
modelDropdown = widgets.Dropdown(
    options=['MAR','RACMO','MERRA2-hybrid'],
    value='MAR',
    description='Model:',
    disabled=False,
)

# dropdown menu for setting model version
models = dict(AA={}, GL={})
# MAR
models['GL']['MAR'] = []
# models['GL']['MAR'].append('MARv3.9-ERA')
# models['GL']['MAR'].append('MARv3.10-ERA')
# models['GL']['MAR'].append('MARv3.11-NCEP')
models['GL']['MAR'].append('MARv3.11-ERA')
models['GL']['MAR'].append('MARv3.11.2-ERA-7.5km')
models['GL']['MAR'].append('MARv3.11.2-ERA-10km')
models['GL']['MAR'].append('MARv3.11.2-ERA-15km')
models['GL']['MAR'].append('MARv3.11.2-ERA-20km')
models['GL']['MAR'].append('MARv3.11.2-NCEP-20km')
# RACMO
models['GL']['RACMO'] = []
# models['GL']['RACMO'].append('RACMO2.3-XGRN11')
# models['GL']['RACMO'].append('RACMO2.3p2-XGRN11')
models['GL']['RACMO'].append('RACMO2.3p2-FGRN055')
# MERRA2-hybrid
models['GL']['MERRA2-hybrid'] = []
# models['GL']['MERRA2-hybrid'].append('GSFC-fdm-v0')
models['GL']['MERRA2-hybrid'].append('GSFC-fdm-v1')
models['AA']['MERRA2-hybrid'] = []
# models['AA']['MERRA2-hybrid'].append('GSFC-fdm-v0')
models['AA']['MERRA2-hybrid'].append('GSFC-fdm-v1')
versionDropdown = widgets.Dropdown(
    options=models[regionDropdown.value][modelDropdown.value],
    value=models[regionDropdown.value][modelDropdown.value][0],
    description='Version:',
    disabled=False,
)

# function for updating the model version
def set_version(sender):
    versionDropdown.options=models[regionDropdown.value][modelDropdown.value]
    versionDropdown.value=models[regionDropdown.value][modelDropdown.value][0]

# watch widgets for changes
regionDropdown.observe(set_version)
modelDropdown.observe(set_version)

# display widgets for setting parameters
widgets.VBox([inputText,directoryText,regionDropdown,
    modelDropdown,versionDropdown])

#### Function to convert ATL11 time variables

In [ ]:
# PURPOSE: convert time from delta seconds into Julian and year-decimal
def convert_delta_time(delta_time, gps_epoch=1198800018.0):
    # calculate gps time from delta_time
    gps_seconds = gps_epoch + delta_time
    time_leaps = SMBcorr.count_leap_seconds(gps_seconds)
    # calculate julian time
    time_julian = 2444244.5 + (gps_seconds - time_leaps)/86400.0
    # convert to calendar date with convert_julian.py
    Y,M,D,h,m,s = SMBcorr.convert_julian(time_julian,FORMAT='tuple')
    # calculate year-decimal time
    time_decimal = SMBcorr.convert_calendar_decimal(Y,M,DAY=D,HOUR=h,MINUTE=m,SECOND=s)
    # return both the Julian and year-decimal formatted dates
    return dict(julian=time_julian, decimal=time_decimal)

#### Function to set the coordinate projection

In [ ]:
# PURPOSE: set the projection parameters based on the region name
def set_projection(REGION):
    if (REGION == 'AA'):
        projection_flag = 'EPSG:3031'
    elif (REGION == 'GL'):
        projection_flag = 'EPSG:3413'
    return projection_flag

#### Run interpolation program for model and ATL11 file

In [ ]:
# read input file
input_file = os.path.expanduser(inputText.value)
field_dict = {None:('delta_time','h_corr','x','y')}
D11 = pc.data().from_h5(input_file, field_dict=field_dict)
# check if running crossover or along-track ATL11
if (D11.h_corr.ndim == 3):
    nseg,ncycle,ncross = D11.shape
else:
    nseg,ncycle = D11.shape
        
# get projection of input coordinates
EPSG = set_projection(regionDropdown.value)

# extract parameters from widgets
base_dir=os.path.expanduser(directoryText.value)
if (modelDropdown.value == 'MAR'):
    match_object=re.match('(MARv\d+\.\d+(.\d+)?)',versionDropdown.value)
    MAR_VERSION=match_object.group(0)
    MAR_REGION=dict(GL='Greenland',AA='Antarctic')[regionDropdown.value]
    # model subdirectories
    SUBDIRECTORY=dict(AA={}, GL={})
    SUBDIRECTORY['GL']['MARv3.9-ERA']=['ERA_1958-2018_10km','daily_10km']
    SUBDIRECTORY['GL']['MARv3.10-ERA']=['ERA_1958-2019-15km','daily_15km']
    SUBDIRECTORY['GL']['MARv3.11-NCEP']=['NCEP1_1948-2020_20km','daily_20km']
    SUBDIRECTORY['GL']['MARv3.11-ERA']=['ERA_1958-2019-15km','daily_15km']
    SUBDIRECTORY['GL']['MARv3.11.2-ERA-7.5km']=['7.5km_ERA5']
    SUBDIRECTORY['GL']['MARv3.11.2-ERA-10km']=['10km_ERA5']
    SUBDIRECTORY['GL']['MARv3.11.2-ERA-15km']=['15km_ERA5']
    SUBDIRECTORY['GL']['MARv3.11.2-ERA-20km']=['20km_ERA5']
    SUBDIRECTORY['GL']['MARv3.11.2-NCEP-20km']=['20km_NCEP1']
    MAR_MODEL=SUBDIRECTORY[regionDropdown.value][versionDropdown.value]
    DIRECTORY=os.path.join(base_dir,'MAR',MAR_VERSION,MAR_REGION,*MAR_MODEL)    
    # variable coordinates
    KWARGS=dict(AA={}, GL={})
    KWARGS['GL']['MARv3.9-ERA'] = dict(XNAME='X10_153',YNAME='Y21_288')
    KWARGS['GL']['MARv3.10-ERA'] = dict(XNAME='X10_105',YNAME='Y21_199')
    KWARGS['GL']['MARv3.11-NCEP'] = dict(XNAME='X12_84',YNAME='Y21_155')
    KWARGS['GL']['MARv3.11-ERA'] = dict(XNAME='X10_105',YNAME='Y21_199')
    KWARGS['GL']['MARv3.11.2-ERA-7.5km'] = dict(XNAME='X12_203',YNAME='Y20_377')
    KWARGS['GL']['MARv3.11.2-ERA-10km'] = dict(XNAME='X10_153',YNAME='Y21_288')
    KWARGS['GL']['MARv3.11.2-ERA-15km'] = dict(XNAME='X10_105',YNAME='Y21_199')
    KWARGS['GL']['MARv3.11.2-ERA-20km'] = dict(XNAME='X12_84',YNAME='Y21_155')
    KWARGS['GL']['MARv3.11.2-NCEP-20km'] = dict(XNAME='X12_84',YNAME='Y21_155')
    MAR_KWARGS=KWARGS[regionDropdown.value][versionDropdown.value]
    # output variable keys for both direct and derived fields
    KEYS = ['zsurf','zfirn','zmelt','zsmb','zaccum']
    # HDF5 longname attributes for each variable
    LONGNAME = {}
    LONGNAME['zsurf'] = "Snow Height Change"
    LONGNAME['zfirn'] = "Snow Height Change due to Compaction"
    LONGNAME['zmelt'] = "Snow Height Change due to Surface Melt"
    LONGNAME['zsmb'] = "Snow Height Change due to Surface Mass Balance"
    LONGNAME['zaccum'] = "Snow Height Change due to Surface Accumulation"
elif (modelDropdown.value == 'RACMO'):
    RACMO_VERSION,RACMO_MODEL=versionDropdown.value.split('-')
    # output variable keys
    KEYS = ['zsurf']
    # HDF5 longname attributes for each variable
    LONGNAME = {}
    LONGNAME['zsurf'] = "Snow Height Change"
elif (modelDropdown.value == 'MERRA2-hybrid'):
    MERRA2_VERSION,=re.findall('GSFC-fdm-(.*?)$',versionDropdown.value)
    # MERRA-2 hybrid directory
    DIRECTORY=os.path.join(base_dir,'MERRA2_hybrid',MERRA2_VERSION)
    MERRA2_REGION = dict(AA='ais',GL='gris')[regionDropdown.value]
    # output variable keys for both direct and derived fields
    KEYS = ['zsurf','zfirn','zsmb']
    # HDF5 longname attributes for each variable
    LONGNAME = {}
    LONGNAME['zsurf'] = "Snow Height Change"
    LONGNAME['zfirn'] = "Snow Height Change due to Compaction"
    LONGNAME['zsmb'] = "Snow Height Change due to Surface Mass Balance"
    
# check if running crossover or along track
if (D11.h_corr.ndim == 3):
    # allocate for output height for crossover data 
    OUTPUT = {}
    for key in KEYS:
        OUTPUT[key] = np.ma.zeros((nseg,ncycle,ncross),fill_value=np.nan)
        OUTPUT[key].mask = np.ones((nseg,ncycle,ncross),dtype=np.bool)
        OUTPUT[key].interpolation = np.zeros((nseg,ncycle,ncross),dtype=np.uint8)
    # for each cycle of ICESat-2 ATL11 data
    for c in range(ncycle):
        # check that there are valid crossovers
        cross = [xo for xo in range(ncross) if
            np.any(np.isfinite(D11.delta_time[:,c,xo]))]
        # for each valid crossing
        for xo in cross:
            # find valid crossovers
            i, = np.nonzero(np.isfinite(D11.delta_time[:,c,xo]))
            # convert from delta time to decimal-years
            tdec = convert_delta_time(D11.delta_time[i,c,xo])['decimal']
            if (modelDropdown.value == 'MAR'):
                # read and interpolate daily MAR outputs
                ZN4 = SMBcorr.interpolate_mar_daily(DIRECTORY, EPSG,
                    MAR_VERSION, tdec, D11.x[i,c,xo], D11.y[i,c,xo],
                    VARIABLE='ZN4', SIGMA=1.5, FILL_VALUE=np.nan, **MAR_KWARGS)
                ZN5 = SMBcorr.interpolate_mar_daily(DIRECTORY, EPSG,
                    MAR_VERSION, tdec, D11.x[i,c,xo], D11.y[i,c,xo],
                    VARIABLE='ZN5', SIGMA=1.5, FILL_VALUE=np.nan, **MAR_KWARGS)
                ZN6 = SMBcorr.interpolate_mar_daily(DIRECTORY, EPSG,
                    MAR_VERSION, tdec, D11.x[i,c,xo], D11.y[i,c,xo],
                    VARIABLE='ZN6', SIGMA=1.5, FILL_VALUE=np.nan, **MAR_KWARGS)
                # set attributes to output for iteration
                OUTPUT['zfirn'].data[i,c,xo] = np.copy(ZN4.data)
                OUTPUT['zfirn'].mask[i,c,xo] = np.copy(ZN4.mask)
                OUTPUT['zfirn'].interpolation[i,c,xo] = np.copy(ZN4.interpolation)
                OUTPUT['zsurf'].data[i,c,xo] = np.copy(ZN6.data)
                OUTPUT['zsurf'].mask[i,c,xo] = np.copy(ZN6.mask)
                OUTPUT['zsurf'].interpolation[i,c,xo] = np.copy(ZN6.interpolation)
                OUTPUT['zmelt'].data[i,c,xo] = np.copy(ZN5.data)
                OUTPUT['zmelt'].mask[i,c,xo] = np.copy(ZN5.mask)
                OUTPUT['zmelt'].interpolation[i,c,xo] = np.copy(ZN5.interpolation)
                # calculate derived fields
                OUTPUT['zsmb'].data[i,c,xo] = ZN6.data - ZN4.data
                OUTPUT['zsmb'].mask[i,c,xo] = ZN4.mask | ZN6.mask
                OUTPUT['zaccum'].data[i,c,xo] = ZN6.data - ZN4.data - ZN5.data
                OUTPUT['zaccum'].mask[i,c,xo] = ZN4.mask | ZN5.mask | ZN6.mask
            elif (modelDropdown.value == 'RACMO'):
                # read and interpolate daily RACMO outputs
                hgtsrf = SMBcorr.interpolate_racmo_daily(base_dir, EPSG,
                    RACMO_MODEL, tdec, D11.x[i,c,xo], D11.y[i,c,xo],
                    VARIABLE='hgtsrf', SIGMA=1.5, FILL_VALUE=np.nan)
                # set attributes to output for iteration
                OUTPUT['zsurf'].data[i,c,xo] = np.copy(hgtsrf.data)
                OUTPUT['zsurf'].mask[i,c,xo] = np.copy(hgtsrf.mask)
                OUTPUT['zsurf'].interpolation[i,c,xo] = np.copy(hgtsrf.interpolation)
            elif (modelDropdown.value == 'MERRA2-hybrid'):
                # read and interpolate 5-day MERRA2-Hybrid outputs
                FAC = SMBcorr.interpolate_merra_hybrid(DIRECTORY, EPSG,
                    MERRA2_REGION, tdec, D11.x[i,c,xo], D11.y[i,c,xo],
                    VERSION=MERRA2_VERSION, VARIABLE='FAC',
                    SIGMA=1.5, FILL_VALUE=np.nan)
                smb = SMBcorr.interpolate_merra_hybrid(DIRECTORY, EPSG,
                    MERRA2_REGION, tdec, D11.x[i,c,xo], D11.y[i,c,xo],
                    VERSION=MERRA2_VERSION, VARIABLE='cum_smb_anomaly',
                    SIGMA=1.5, FILL_VALUE=np.nan)
                height = SMBcorr.interpolate_merra_hybrid(DIRECTORY, EPSG,
                    MERRA2_REGION, tdec, D11.x[i,c,xo], D11.y[i,c,xo],
                    VERSION=MERRA2_VERSION, VARIABLE='height',
                    SIGMA=1.5, FILL_VALUE=np.nan)
                # set attributes to output for iteration
                OUTPUT['zfirn'].data[i,c,xo] = np.copy(FAC.data)
                OUTPUT['zfirn'].mask[i,c,xo] = np.copy(FAC.mask)
                OUTPUT['zfirn'].interpolation[i,c,xo] = np.copy(FAC.interpolation)
                OUTPUT['zsurf'].data[i,c,xo] = np.copy(height.data)
                OUTPUT['zsurf'].mask[i,c,xo] = np.copy(height.mask)
                OUTPUT['zsurf'].interpolation[i,c,xo] = np.copy(height.interpolation)
                OUTPUT['zsmb'].data[i,c,xo] = np.copy(smb.data)
                OUTPUT['zsmb'].mask[i,c,xo] = np.copy(smb.mask)
                OUTPUT['zsmb'].interpolation[i,c,xo] = np.copy(smb.interpolation)
else:        
    # allocate for output height for along-track data 
    OUTPUT = {}
    for key in KEYS:
        OUTPUT[key] = np.ma.zeros((nseg,ncycle),fill_value=np.nan)
        OUTPUT[key].mask = np.ones((nseg,ncycle),dtype=np.bool)
        OUTPUT[key].interpolation = np.zeros((nseg,ncycle),dtype=np.uint8)    
    # check that there are valid elevations
    cycle = [c for c in range(ncycle) if
        np.any(np.isfinite(D11.delta_time[:,c]))]
    # for each valid cycle of ICESat-2 ATL11 data
    for c in cycle:
        # find valid elevations
        i, = np.nonzero(np.isfinite(D11.delta_time[:,c]))
        # convert from delta time to decimal-years
        tdec = convert_delta_time(D11.delta_time[i,c])['decimal']
        if (modelDropdown.value == 'MAR'):
            # read and interpolate daily MAR outputs
            ZN4 = SMBcorr.interpolate_mar_daily(DIRECTORY, EPSG,
                MAR_VERSION, tdec, D11.x[i,c], D11.y[i,c], VARIABLE='ZN4',
                SIGMA=1.5, FILL_VALUE=np.nan, **MAR_KWARGS)
            ZN5 = SMBcorr.interpolate_mar_daily(DIRECTORY, EPSG,
                MAR_VERSION, tdec, D11.x[i,c], D11.y[i,c], VARIABLE='ZN5',
                SIGMA=1.5, FILL_VALUE=np.nan, **MAR_KWARGS)
            ZN6 = SMBcorr.interpolate_mar_daily(DIRECTORY, EPSG,
                MAR_VERSION, tdec, D11.x[i,c], D11.y[i,c], VARIABLE='ZN6',
                SIGMA=1.5, FILL_VALUE=np.nan, **MAR_KWARGS)
            # set attributes to output for iteration
            OUTPUT['zfirn'].data[i,c] = np.copy(ZN4.data)
            OUTPUT['zfirn'].mask[i,c] = np.copy(ZN4.mask)
            OUTPUT['zfirn'].interpolation[i,c] = np.copy(ZN4.interpolation)
            OUTPUT['zsurf'].data[i,c] = np.copy(ZN6.data)
            OUTPUT['zsurf'].mask[i,c] = np.copy(ZN6.mask)
            OUTPUT['zsurf'].interpolation[i,c] = np.copy(ZN6.interpolation)
            OUTPUT['zmelt'].data[i,c] = np.copy(ZN5.data)
            OUTPUT['zmelt'].mask[i,c] = np.copy(ZN5.mask)
            OUTPUT['zmelt'].interpolation[i,c] = np.copy(ZN5.interpolation)
            # calculate derived fields
            OUTPUT['zsmb'].data[i,c] = ZN6.data - ZN4.data
            OUTPUT['zsmb'].mask[i,c] = ZN4.mask | ZN6.mask
            OUTPUT['zaccum'].data[i,c] = ZN6.data - ZN4.data - ZN5.data
            OUTPUT['zaccum'].mask[i,c] = ZN4.mask | ZN5.mask | ZN6.mask            
        elif (modelDropdown.value == 'RACMO'):
            # read and interpolate daily RACMO outputs
            hgtsrf = SMBcorr.interpolate_racmo_daily(base_dir, EPSG,
                RACMO_MODEL, tdec, D11.x[i,c], D11.y[i,c],
                VARIABLE='hgtsrf', SIGMA=1.5, FILL_VALUE=np.nan)
            # set attributes to output for iteration
            OUTPUT['zsurf'].data[i,c] = np.copy(hgtsrf.data)
            OUTPUT['zsurf'].mask[i,c] = np.copy(hgtsrf.mask)
            OUTPUT['zsurf'].interpolation[i,c] = np.copy(hgtsrf.interpolation)
        elif (modelDropdown.value == 'MERRA2-hybrid'):
            # read and interpolate 5-day MERRA2-Hybrid outputs
            FAC = SMBcorr.interpolate_merra_hybrid(DIRECTORY, EPSG,
                MERRA2_REGION, tdec, D11.x[i,c], D11.y[i,c],
                VERSION=MERRA2_VERSION, VARIABLE='FAC',
                SIGMA=1.5, FILL_VALUE=np.nan)
            smb = SMBcorr.interpolate_merra_hybrid(DIRECTORY, EPSG,
                MERRA2_REGION, tdec, D11.x[i,c], D11.y[i,c],
                VERSION=MERRA2_VERSION, VARIABLE='cum_smb_anomaly',
                SIGMA=1.5, FILL_VALUE=np.nan)
            height = SMBcorr.interpolate_merra_hybrid(DIRECTORY, EPSG,
                MERRA2_REGION, tdec, D11.x[i,c], D11.y[i,c],
                VERSION=MERRA2_VERSION, VARIABLE='height',
                SIGMA=1.5, FILL_VALUE=np.nan)
            # set attributes to output for iteration
            OUTPUT['zfirn'].data[i,c] = np.copy(FAC.data)
            OUTPUT['zfirn'].mask[i,c] = np.copy(FAC.mask)
            OUTPUT['zfirn'].interpolation[i,c] = np.copy(FAC.interpolation)
            OUTPUT['zsurf'].data[i,c] = np.copy(height.data)
            OUTPUT['zsurf'].mask[i,c] = np.copy(height.mask)
            OUTPUT['zsurf'].interpolation[i,c] = np.copy(height.interpolation)
            OUTPUT['zsmb'].data[i,c] = np.copy(smb.data)
            OUTPUT['zsmb'].mask[i,c] = np.copy(smb.mask)
            OUTPUT['zsmb'].interpolation[i,c] = np.copy(smb.interpolation)
                
# verify mask values
for key in KEYS:
    OUTPUT[key].mask |= (OUTPUT[key].data == OUTPUT[key].fill_value) | \
        np.isnan(OUTPUT[key].data)
    OUTPUT[key].data[OUTPUT[key].mask] = OUTPUT[key].fill_value

#### Create plot showing elevation change and total modeled height change

In [ ]:
if (D11.h_corr.ndim == 2):
    fig,(ax1,ax2) = plt.subplots(ncols=2)
    c1,c2 = (0,1)
    i, = np.nonzero(np.isfinite(D11.h_corr[:,c1]) & np.isfinite(D11.h_corr[:,c2]) & 
        (~OUTPUT['zsurf'].mask[:,c1]) & (~OUTPUT['zsurf'].mask[:,c2]))
    # height change from ICESat-2
    ax1.scatter(D11.x[i,c1],D11.y[i,c1],
        c=D11.h_corr[i,c2]-D11.h_corr[i,c1],
        vmin=-5,vmax=0.5,s=0.5)
    ax1.set_title = 'ATL11'
    # firn height change
    ax2.scatter(D11.x[i,c1],D11.y[i,c1],
        c=OUTPUT['zsurf'][i,c2]-OUTPUT['zsurf'][i,c1],
        vmin=-5,vmax=0.5,s=0.5)
    ax2.set_title = versionDropdown.value
    plt.show()

#### Append data to input ATL11 HDF5 file

In [ ]:
# append input HDF5 file with new firn model outputs
fileID = h5py.File(os.path.expanduser(inputText.value),'a')
fileID.create_group(versionDropdown.value)
h5 = {}
for key in KEYS:
    val = '{0}/{1}'.format(versionDropdown.value,key)
    h5[key] = fileID.create_dataset(val, OUTPUT[key].shape,
        data=OUTPUT[key], dtype=OUTPUT[key].dtype,
        compression='gzip', fillvalue=OUTPUT[key].fill_value)
    h5[key].attrs['units'] = "m"
    h5[key].attrs['long_name'] = LONGNAME[key]
    h5[key].attrs['coordinates'] = "../delta_time ../latitude ../longitude"
    h5[key].attrs['model'] = versionDropdown.value
# close the output HDF5 file
fileID.close()